In [1]:
from pyspark import SparkContext
sc = SparkContext("local[*]")

In [116]:
rdd = sc.textFile("Posts.xml")

In [117]:
import re

rdd_text = rdd.map(lambda x: [get_field(x, "Body")])#re.search("Body=\"(.*?)\"", x))
rdd_text = rdd_text.filter(lambda x: x is not None)



In [17]:
rdd_tags = rdd.map(lambda x: re.search("Tags=\"(.*?)\"", x))
rdd_tags = rdd_tags.filter(lambda x: x is not None).map(lambda x: x.group(1))
rdd_tags = rdd_tags.map(lambda x: remove_tag_brackets(x))
rdd_tags.take(5)

[['team-fortress-2'],
 ['starcraft-2', 'starcraft-protoss'],
 ['steam', 'source-engine'],
 ['the-secret-of-monkey-island'],
 ['backwards-compatibility', 'windows-xp', 'sam-and-max']]

In [16]:
def remove_tag_brackets(text):
    text = text.replace("&gt;&lt;", " ")
    text = text.replace("&gt;", "")
    text = text.replace("&lt;", "")
    text = text.split()
    return text


In [11]:
"".replace?


Object `replace` not found.


In [ ]:
"".replace

In [25]:
rdd_text.take(5)

['&lt;p&gt;Now that the Engineer update has come, there will be lots of Engineers building up everywhere.  How should this best be handled?&lt;/p&gt;&#xA;',
 "&lt;p&gt;I know I can create a Warp Gate and teleport to Pylons, but I have no idea how to make Warp Prisms or know if there's any other unit capable of transporting.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I would in particular like this to built remote bases in 1v1&lt;/p&gt;&#xA;",
 "&lt;p&gt;Steam won't let me have two instances running with the same user logged in.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Does that mean I cannot run a dedicated server on a PC (for example, for Left 4 Dead 2) &lt;em&gt;and&lt;/em&gt; play from another machine?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Is there a way to run the dedicated server without running Steam? Is there a configuration option I'm missing?&lt;/p&gt;&#xA;",
 '&lt;p&gt;When I get to the insult sword-fighting stage of The Secret of Monkey Island, do I have to learn every single insult and comeback in order to beat

In [23]:
def get_field(line, field_name):
    found = re.search("{}=\"(.*?)\"".format(field_name), line)
    if found:
        return found.group(1)

In [147]:
ARBITRATRY_CUTOFF_VALUE = 2

def get_relevant_fields(line):
    return [get_field(line, "Body"),
            #get_field(line, "Tags"),
            get_field(line, "Score")]


def any_missing(row):
    return row[0] and row[1]# and row[2]


def preprocess_scores(score):
    score = int(score)
    return 1 if score > 0 else 0


def preprocess_tags(tags):
    tags = tags.replace("&gt;&lt;", " ")
    tags = tags.replace("&gt;", "")
    tags = tags.replace("&lt;", "")
    tags = tags.split()
    return len(tags)


def remove_stop_words(counter):
    stop_words = set(["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "arent", 
                     "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", 
                     "cant", "cannot", "could", "couldnt", "did", "didnt", "do", "does", "doesnt", "doing", "dont", "down", 
                     "during", "each", "few", "for", "from", "further", "had", "hadnt", "has", "hasnt", "have", "havent", 
                     "having", "he", "hed", "hell", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", 
                     "his", "how", "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "isnt", "it", "its", 
                     "its", "itself", "lets", "me", "more", "most", "mustnt", "my", "myself", "no", "nor", "not", "of", 
                     "off", "on", "once", "only", "or", "other", "ought", "our", "ours 	ourselves", "out", "over", "own", 
                     "same", "shant", "she", "shed", "shell", "shes", "should", "shouldnt", "so", "some", "such", "than", 
                     "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "there", "theres", "these", 
                     "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through", "to", "too", "under", 
                     "until", "up", "very", "was", "wasnt", "we", "wed", "well", "were", "weve", "were", "werent", "what", 
                     "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why", "whys", 
                     "with", "wont", "would", "wouldnt", "you", "youd", "youll", "youre", "youve", "your", "yours", 
                     "yourself", "yourselves"])
    words = set(counter.keys())
    for word in words:
        if word in stop_words:
            del counter[word]
    return counter


def preprocess_text(text):
    # remove code snippets
    text = re.sub("&lt;pre&gt;&lt;code&gt;.*?&lt;/pre&gt;&lt;/code&gt;", "", text)
    
    # remove html tags
    text = re.sub("&lt;.*?&gt;", "", text)
    
    #remove whatever this is
    text = re.sub("&#xA;", "", text)
    
    # remove links
    text = re.sub("https?://.*?\s|$", "", text)
    
    # remove symbols
    text = re.sub("[^a-zA-Z\s]", "", text)
    
    # normalize whitespace
    text = re.sub("\s+", " ", text)
    
    # lowercase everything
    text = text.lower()
    
    # split text into words
    split = text.split()

    # count words
    counter = Counter(split)
    
    # remove stop words
    counter = remove_stop_words(counter)
    
    return counter

def preprocess_all(row):
    return (preprocess_text(row[0]), preprocess_scores(row[1]), 1)


assert preprocess_text("&lt;pre&gt;&lt;code&gt;cprintf('hello world!');&lt;/pre&gt;&lt;/code&gt;"
                       "https://google.com "
                       "boulder leave it's it it?&lt;pre&gt;&lt;/pre&gt;") == Counter(["leave", "boulder"])


def get_vector_words(counter):
    for key in list(counter.keys()):
        if counter[key] < ARBITRATRY_CUTOFF_VALUE:
            del counter[key]
    return list(counter.keys())


In [166]:
rdd_all = rdd.map(get_relevant_fields)
rdd_all = rdd_all.filter(any_missing)
rdd_all = rdd_all.map(preprocess_all)
#rdd_pos = rdd_all.filter(lambda x: x[1] == 1)
#rdd_pos = rdd_pos.reduce(lambda x,y: (x[0]+y[0], x[1], x[2]+y[2]))
#rdd_neg = rdd_all.filter(lambda x: x[1] == 0)
#rdd_neg = rdd_neg.reduce(lambda x,y: (x[0]+y[0], x[1], x[2]+y[2]))

#count = rdd_neg.count()
#train_count = count // 4 * 3
#test_count = count - train_count

#rdd_pos = rdd_pos.sample(False, 0.1)

reduced = rdd_all.reduce(lambda x,y: (x[0]+y[0], x[1], x[2]+y[2]))
vector_words = get_vector_words(reduced[0])
#print(vector_words)
n = len(vector_words)

def create_vector(row):
    vector = []
    index = []
    length = 0
    counter = row[0]
    cat = float(row[1])
    for i in range(n):
        word = vector_words[i]
        if word in counter.keys():
            vector.append(float(counter[word]))
            index.append(i)
            length += 1
    return LabeledPoint(cat, SparseVector(n, index, vector))

rdd_all = rdd_all.map(create_vector)

rdd_train, rdd_test = rdd_all.randomSplit(weights=[0.7, 0.3])

#print(count)
#print(rdd_neg)
print(rdd_train)
#print(rdd_pos)

PythonRDD[194] at RDD at PythonRDD.scala:53


In [157]:
from collections import Counter

In [81]:
c = Counter(["a", "a", "B"])


In [161]:
rdd_train.take(5)

[LabeledPoint(1.0, (10028,[0,1,2,3,4,5,6,7],[1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0])),
 LabeledPoint(1.0, (10028,[1,8,9,10,11,12,13,14,15,16,17,18,19,20],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(1.0, (10028,[1,4,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36],[2.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(1.0, (10028,[5,6,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63],[3.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])),
 LabeledPoint(1.0, (10028,[1,64,65,66,67,68,69,70,71,72,73,74],[1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0]))]

In [167]:
svm = SVMWithSGD.train(rdd_train, iterations=10)

In [126]:
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

In [184]:
predictionAndLabel = rdd_test.map(lambda x: (float(x.label), float(svm.predict(x.features))))


In [194]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics 

metrics = BinaryClassificationMetrics(predictionAndLabel)

In [193]:
predictionAndLabel = rdd_test.map(lambda x: (float(x.label), float(0)))


In [172]:
metrics

In [175]:
dir(metrics)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_java_model',
 '_sc',
 'areaUnderPR',
 'areaUnderROC',
 'call',
 'unpersist']

In [196]:
metrics.areaUnderPR

0.0

In [182]:
predictionAndLabel.take(5)

[(1.0, 1), (1.0, 1), (1.0, 1), (1.0, 1), (1.0, 1)]